In [1]:
from cerebralcortex.util.helper_methods import get_study_names
sn = get_study_names("/home/jupyter/cc3_conf/")
print(sn)
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, FloatType, TimestampType, IntegerType
from pyspark.sql.functions import minute, second, mean, window
from pyspark.sql import functions as F
import numpy as np

from cerebralcortex import Kernel
CC = Kernel("/home/jupyter/cc3_conf/", study_name='nu')

CC.search_stream('accel')

stream_id = 'org.md2k.autosense.accel.activity.10seconds'

def adc_to_acceleration(x):
    if x>=.5:
        return 1
    return 0
transform_udf = F.udf(adc_to_acceleration, IntegerType())

aclx = CC.get_stream(stream_id)

acl_average = aclx.compute_average(windowDuration=60,startTime='0 seconds')

acl_activity = acl_average.withColumn('activity',transform_udf('activity'))

acl_activity.show()

from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor,ModuleMetadata

acl_activity.printSchema()

stream_name = 'org.md2k.autosense.accel.activity.60seconds'
stream_metadata = Metadata()
stream_metadata.set_name(stream_name).set_description("Chest Accelerometer Activity in 60 second windows") \
    .add_dataDescriptor(
    DataDescriptor().set_name("activity").set_type("integer").set_attribute("description", \
    "accelerometer activity").set_attribute('0','Stationery').set_attribute('1','Non Stationery')) \
    .add_dataDescriptor(
    DataDescriptor().set_name("window").set_type("struct").set_attribute("description", \
    "window start and end time in UTC").set_attribute('start', \
    'start of 1 minute window').set_attribute('end','end of 1 minute window')) \
    .add_module(
    ModuleMetadata().set_name("fourtytwo/mullah/cc3/60_seconds_activity.ipynb").set_attribute("attribute_key", "attribute_value").set_author(
        "Md Azim Ullah", "mullah@memphis.edu"))
stream_metadata.is_valid()

stream_metadata

data = acl_activity._data

ds = DataStream(data=data,metadata=stream_metadata)
ds.printSchema()

CC.save_stream(ds)

['admin', 'dartmouth_sobc', 'default', 'demo', 'jhu_cocaine', 'md2k_aa_rice', 'md2k_labtest', 'md2k_ses_utah', 'memphis-test', 'memphis_test_study', 'moffitt', 'moffitt-test', 'moods', 'moral', 'mperf', 'mperf-alabsi', 'mperf-buder', 'mperf-mit-ll', 'mperf-test', 'northwestern_smoking', 'nu', 'opioid_study', 'osu', 'rice', 'robas', 'robas_study', 'sobclab', 'utah', 'utah_p01', 'vermont', 'vermont_smoking']
+--------------------+-------+--------------------+--------+-------------------+-------------------+
|                user|version|              window|activity|          timestamp|          localtime|
+--------------------+-------+--------------------+--------+-------------------+-------------------+
|bfc6b51c-2899-333...|      2|[2019-03-28 21:36...|       0|2019-03-28 21:36:00|2019-03-28 15:36:00|
|bfc6b51c-2899-333...|      2|[2019-03-29 14:37...|       0|2019-03-29 14:37:00|2019-03-29 08:37:00|
|bfc6b51c-2899-333...|      2|[2019-03-30 17:00...|       0|2019-03-30 17:00:00|2019-

True

In [2]:
ds.count()

727475

In [ ]:
stream_id = 'org.md2k.autosense.accel.activity.60seconds'
acl_activity = CC.get_stream(stream_id)

In [ ]:
df = acl_activity.select('activity').toPandas()

In [ ]:
df.columns

In [ ]:
df['activity'][:100]

In [ ]:
from pyspark.sql.functions import col, countDistinct,sumDistinct
acl_activity.agg(sum(col("activity")).alias("count")).show()

In [ ]:
def cross_entropy(p, q):
	return -sum([p[i]*np.log2(q[i]) for i in range(len(p)) if q[i]>0])

In [ ]:
for i in range(20):
    print(cross_entropy(np.random.randn(10),np.random.randn(10)))